In [1]:
import torch
import torchvision

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def set_seeds(seed: int=42):
    torch.manual_seed(seed)
    torch.random.manual_seed(seed)

In [4]:
import os
import zipfile
from pathlib import Path
import requests

def get_data(source:str, destination: str, remove_source: bool = True) -> Path:
    data_path = Path("data")
    image_path = data_path / destination

    if image_path.is_dir():
        print("Dataset sudah ada")
    else:
        print("Dataset belum ada,mendownload dataset...")

        image_path.mkdir(parents=True,exist_ok=True)

        target_file = Path(source).name

        with open(data_path / target_file, "wb") as f:
            res = requests.get(source)
            f.write(res.content)

        with zipfile.ZipFile(data_path / target_file, "r") as zf:
            print("Mengektrak data...")
            zf.extractall(image_path)
        
        if remove_source:
                os.remove(data_path / target_file)

    return image_path

In [5]:
image_path  = get_data(
    source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip",
    destination="pizza_steak_sushi"
)

image_path

Dataset sudah ada


WindowsPath('data/pizza_steak_sushi')

In [6]:
from torchvision.transforms import v2
from torchvision.transforms.functional import InterpolationMode

manual_transforms = v2.Compose(
    [
        v2.ToImage(),
        v2.Resize(size=(256, 256), interpolation=InterpolationMode.BICUBIC),
        v2.CenterCrop(size=(224, 224)),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [7]:
from going_modular.data_setup import create_dataloaders

train_dir = image_path / "train"
test_dir = image_path / "test"
BATCH_SIZE = 32

train_dataloader, test_dataloader, class_names  = create_dataloaders(train_dir,test_dir,manual_transforms,manual_transforms,BATCH_SIZE)

train_dataloader, test_dataloader, class_names

(<torch.utils.data.dataloader.DataLoader at 0x2287106cc40>,
 ['pizza', 'steak', 'sushi'])

In [8]:
from torchvision.models.efficientnet import EfficientNet_B0_Weights

weights = EfficientNet_B0_Weights.DEFAULT

auto_transforms = weights.transforms()

In [9]:
train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir, test_dir, auto_transforms, auto_transforms, BATCH_SIZE
)

print(f"Automatically Created Transform {auto_transforms}")
train_dataloader, test_dataloader, class_names

Automatically Created Transform ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x2287106fdc0>,
 ['pizza', 'steak', 'sushi'])

In [10]:
from torchvision.models.efficientnet import efficientnet_b0, EfficientNet_B0_Weights

model = efficientnet_b0(weights=EfficientNet_B0_Weights).to(device)

c:\Coding\python-projects\pytorch-db\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
from torchinfo import summary

summary(
    model,
    input_size=(BATCH_SIZE, 3, 224, 224),
    col_names=["input_size", "num_params", "trainable"],
)

Layer (type:depth-idx)                                  Input Shape               Param #                   Trainable
EfficientNet                                            [32, 3, 224, 224]         --                        True
├─Sequential: 1-1                                       [32, 3, 224, 224]         --                        True
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         --                        True
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         864                       True
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        64                        True
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        --                        --
│    └─Sequential: 2-2                                  [32, 32, 112, 112]        --                        True
│    │    └─MBConv: 3-4                                 [32, 32, 112, 112]        1,448      

In [28]:
# for param in model.features.parameters():
#     param.requires_grad = False

In [29]:
summary(
    model,
    input_size=(BATCH_SIZE, 3, 224, 224),
    col_names=["input_size", "num_params", "trainable"],
)

Layer (type:depth-idx)                                  Input Shape               Param #                   Trainable
EfficientNet                                            [32, 3, 224, 224]         --                        Partial
├─Sequential: 1-1                                       [32, 3, 224, 224]         --                        False
│    └─Conv2dNormActivation: 2-1                        [32, 3, 224, 224]         --                        False
│    │    └─Conv2d: 3-1                                 [32, 3, 224, 224]         (864)                     False
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        --                        --
│    └─Sequential: 2-2                                  [32, 32, 112, 112]        --                        False
│    │    └─MBConv: 3-4                                 [32, 32, 112, 112]        (1,

In [30]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [31]:
from torch import nn

set_seeds(42)

model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names), bias=True)
).to(device)
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=True)
  (1): Linear(in_features=1280, out_features=3, bias=True)
)

In [32]:
summary(model, input_size=(BATCH_SIZE, 3, 224, 224),col_names=["output_size","num_params","trainable"], row_settings=["var_names"])

Layer (type (var_name))                                      Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [32, 3]                   --                        Partial
├─Sequential (features)                                      [32, 1280, 7, 7]          --                        False
│    └─Conv2dNormActivation (0)                              [32, 32, 112, 112]        --                        False
│    │    └─Conv2d (0)                                       [32, 32, 112, 112]        (864)                     False
│    │    └─BatchNorm2d (1)                                  [32, 32, 112, 112]        (64)                      False
│    │    └─SiLU (2)                                         [32, 32, 112, 112]        --                        --
│    └─Sequential (1)                                        [32, 16, 112, 112]        --                        False
│    │    └─MBConv (0)                       

In [33]:
from torchmetrics import Accuracy

loss_fn = nn.CrossEntropyLoss()
acc_fn = Accuracy(task="multiclass",num_classes=len(class_names)).to(device)
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)

In [34]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()

In [35]:
from going_modular.engine import train_model 


## TODO: buat func trainmodel sendiri,panggil func step saja (cache problem, atau pass writer saja ke func train_model)
results = train_model(model,train_dataloader,test_dataloader,loss_fn,acc_fn,optimizer,writer,device,10)


Epoch: 0


Train Loss: 0.0780 | Train Acc: 0.9844
Test Loss: 0.1364 | Test Acc: 0.9400

Epoch: 1


Train Loss: 0.0470 | Train Acc: 0.9822
Test Loss: 0.1348 | Test Acc: 0.9400

Epoch: 2


Train Loss: 0.0354 | Train Acc: 0.9911
Test Loss: 0.1374 | Test Acc: 0.9400

Epoch: 3


Train Loss: 0.2919 | Train Acc: 0.9822
Test Loss: 0.1464 | Test Acc: 0.9467

Epoch: 4


...

Train Loss: 0.0385 | Train Acc: 0.9889
Test Loss: 0.1124 | Test Acc: 0.9600

Epoch: 6


Train Loss: 0.0966 | Train Acc: 0.9889
Test Loss: 0.1151 | Test Acc: 0.9600

Epoch: 7


Train Loss: 0.2665 | Train Acc: 0.9711
Test Loss: 0.1166 | Test Acc: 0.9600

Epoch: 8


Train Loss: 0.0401 | Train Acc: 0.9933
Test Loss: 0.1287 | Test Acc: 0.9400

Epoch: 9


Train Loss: 0.0113 | Train Acc: 0.9978
Test Loss: 0.1214 | Test Acc: 0.9600
